In [1]:
import os
import time
import random
import warnings
import typing as tp
import pathlib
from contextlib import contextmanager

import cv2
import librosa

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import IPython.display as ipd

import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, models, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [2]:
import python_speech_features as psf
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
data_path = pathlib.Path("../data")
audios_path = data_path / "all_audio_resampled"

In [4]:
train = pd.read_csv(data_path/'Train.csv')
train_extra = pd.read_csv(data_path/'train_add.csv')
train_extra_2 = pd.read_csv(data_path/'train_add_20201029.csv')

label2code = {word: idx for idx, word in enumerate(train.label.unique().tolist())}
code2label = {v:k for k,v in label2code.items()}

In [5]:
def get_image_path(audio_path):
    file_name = audio_path.split("/")[-1].split(".")[0]
    ip = str(audios_path.resolve() / f"{file_name}.wav")
    return ip

train["image_fn"] = train.fn.apply(get_image_path)
train_extra["image_fn"] = train_extra.fn.apply(get_image_path)
train_extra_2["image_fn"] = train_extra_2.fn.apply(get_image_path)

In [6]:
train_df = pd.concat([train, train_extra, train_extra_2], axis=0).reset_index(drop=True)

In [7]:
### build validation that includes all classes:


vcs = train_df.label.value_counts()

## possible schema:
# 25+ - take 3
# 12-25 - take 2
# 12- - take 1

def num_for_val(num_examples):
    if num_examples >= 25:
        return 3
    if num_examples >= 12:
        return 2
    return 1

train_df["num_examples"] = train_df.label.map(vcs.to_dict())
train_df["num_for_val"] = train_df.num_examples.apply(num_for_val)

random.seed(12)
train_df_new = pd.DataFrame()
for label in train_df.label.unique():
    tmp = train_df.loc[train_df.label == label].copy()
    tmp["dummy"] = tmp.label.apply(lambda _: random.random())
    tmp.sort_values(by="dummy", inplace=True)
    tmp["rank"] = range(tmp.shape[0])
    train_df_new = pd.concat([train_df_new, tmp])

train_df_new.reset_index(drop=True, inplace=True)
train_df_new["val_subset"] = train_df_new.num_for_val > train_df_new["rank"]
train_df_new.drop("dummy", axis=1, inplace=True)

In [8]:
assert (
    set(train_df_new.loc[train_df_new.val_subset].label.unique()) == 
    set(train_df_new.loc[~train_df_new.val_subset].label.unique())
)

In [9]:
train_df_new["val_fold"] = train_df_new["rank"] // train_df_new.num_for_val

In [10]:
for f in range(10):
    print(f, (train_df_new.val_fold == f).sum())

0 451
1 451
2 451
3 430
4 413
5 410
6 407
7 391
8 381
9 337


In [11]:
N_FOLDS = 3
train_folds = list()
val_folds = list()

for i in range(N_FOLDS):
    valf = train_df_new.loc[train_df_new.val_fold == i].copy()
    trf = train_df_new.loc[train_df_new.val_fold != i].copy()
    
    train_folds.append(trf)
    val_folds.append(valf)

In [12]:
SR = 22050

In [13]:
from reproduce.src.data_processing import new_generate_spec, new_build_image, normalize, MEAN, STD
from reproduce.src.transforms import *

In [14]:
class AudioConfig:
    n_fft = 512
    hop_size = 32
    pad_center = True
    trim = True
    max_len_sec = 2.6
    sr = 22050
    img_size = 299
    
conf = AudioConfig()

In [15]:
aug_noise = AddNoise(0, 0.07)
# aug_ts = TimeStretch((0.5, 2))
aug_pitch = PitchShift((-5, 5), sr=conf.sr)

train_transforms = UseWithProb(
    OneOf([
        aug_noise,
        aug_pitch
    ]),
    prob=0.5
)

In [16]:
class SpectrogramDataset(data.Dataset):
    def __init__(
        self,
        file_list: tp.List[tp.List[str]],
        config,
        transform=None,
        normalize=True
    ):
        self.file_list = file_list  # list of list: [file_path, label]
        self.transform = transform
        self.normalize = normalize
        self.config = config

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx: int):
        fn, word = self.file_list[idx]
        audio, _ = librosa.core.load(fn, sr=SR)

        if self.transform:
            audio = self.transform(audio)

        image = new_build_image(audio, self.config)
        
        if self.normalize:
            norm_image = normalize(np.array(image), mean=MEAN, std=STD)
        else:
            norm_image = image
        
        return np.moveaxis(norm_image, 2, 0), label2code[word]
    
    
class SpectrogramTestDataset(data.Dataset):
    def __init__(
        self,
        file_list: tp.List[tp.List[str]],
        config,
        transform=None,
        normalize=True
    ):
        self.file_list = file_list  # list of list: [file_path, label]
        self.transform = transform
        self.normalize = normalize
        self.config = config

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx: int):
        fn, word = self.file_list[idx]
        audio, _ = librosa.core.load(fn, sr=SR)

        if self.transform:
            audio = self.transform(audio)

        image = new_build_image(audio, self.config)
        
        if self.normalize:
            norm_image = normalize(np.array(image), mean=MEAN, std=STD)
        else:
            norm_image = image
        
        return np.moveaxis(norm_image, 2, 0)

In [17]:
sdf_train_list = [
    SpectrogramDataset(t[["image_fn", "label"]].values.tolist(), conf,
                       transform=train_transforms, normalize=True)
    for t in train_folds
]

sdf_val_list = [
    SpectrogramDataset(v[["image_fn", "label"]].values.tolist(), conf, normalize=True)
    for v in val_folds
]

for s in sdf_train_list:
    print(len(s))
    
print("=========================")

for s in sdf_val_list:
    print(len(s))

4258
4258
4258
451
451
451


In [18]:
def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [19]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def train_mixup_epoch(log_interval, mixup_prob, model, device, criterion, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        use_mixup = False
        if random.random() < mixup_prob:
            use_mixup = True
        data, target = data.type(torch.FloatTensor).to(device), target.to(device)
        
        if use_mixup:
            data, y_a, y_b, lam = mixup_data(data, target)
        optimizer.zero_grad()
        output = model(data)
        if use_mixup:
            loss = mixup_criterion(criterion, output, y_a, y_b, lam) #criterion(output, target)
        else:
            loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return loss.item()

            
def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.type(torch.FloatTensor).to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    accuracy = 100. * correct / len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

    return test_loss, accuracy

In [20]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)


def train_on_fold(
    model,
    train_loader,
    val_loader,
    fold,
    criterion,
    optimizer,
    lr_scheduler,
    results_folder,
    epochs
):
    seed_dict = {
        0: 9,
        1: 99,
        2: 999
    }
    set_seed(seed_dict[fold])

    t0 = time.time()

    best_loss = 1e5
    best_acc = 0

    max_patience = 20
    patience = 0

    train_loss_hist = list()
    val_loss_hist = list()
    val_acc_hist = list()

    save_each_epoch = False

    for ep in range(65):
        train_loss = train_mixup_epoch(1e10, 0.667, model, device, criterion, train_loader, optimizer, ep)
        cur_loss, cur_acc = test(model, device, criterion, val_loader)

        train_loss_hist.append(train_loss)
        val_loss_hist.append(cur_loss)
        val_acc_hist.append(cur_acc)

        if save_each_epoch:
            torch.save(model.state_dict(), f"tmp/{results_folder}/model_ep_{fold}_{ep}.pth")

        if cur_loss < best_loss:
            torch.save(model.state_dict(), f"tmp/{results_folder}/best_run_{fold}.pth")
            best_loss = cur_loss
            best_acc = cur_acc
            patience = 0

        lr_scheduler.step(cur_loss) 
        print("Training so far {} minutes".format((time.time() - t0) / 60))
        print("="*20)
        
    for ep in range(10):
        train_loss = train_mixup_epoch(1e10, 0.0, model, device, criterion, train_loader, optimizer, ep)
        cur_loss, cur_acc = test(model, device, criterion, val_loader)

        train_loss_hist.append(train_loss)
        val_loss_hist.append(cur_loss)
        val_acc_hist.append(cur_acc)

        if save_each_epoch:
            torch.save(model.state_dict(), f"tmp/{results_folder}/model_ep_{fold}_{ep}.pth")

        if cur_loss < best_loss:
            torch.save(model.state_dict(), f"tmp/{results_folder}/best_run_{fold}.pth")
            best_loss = cur_loss
            best_acc = cur_acc
            patience = 0

        lr_scheduler.step(cur_loss)
        print("Training so far {} minutes".format((time.time() - t0) / 60))
        print("="*20)

    print("time spent training: {} minutes".format((time.time() - t0) / 60))
    print("BEST LOSS:", best_loss)
    print("BEST ACC:", best_acc)
    
    model.load_state_dict(torch.load(f"tmp/{results_folder}/best_run_{fold}.pth"))
    
    return model


def predict_on_val(
    model,
    val_loader,
    device="cuda"
):
    sfm = nn.Softmax()
    predictions = list()
    for batch_idx, (inputs, _) in enumerate(val_loader):
        inputs = inputs.type(torch.FloatTensor).to(device)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            predictions.append(sfm(outputs))

    predictions = np.concatenate([t.cpu().numpy() for t in predictions])
    preds = np.argmax(predictions, axis=1)
    return preds


def get_val_outputs(model, test_loader, device="cuda"):
    outputs_list = list()
    for batch_idx, (inputs, target) in enumerate(test_loader):
        inputs = inputs.type(torch.FloatTensor).to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            outputs_list.append(outputs)
    outputs_list = np.concatenate([t.cpu().numpy() for t in outputs_list])
    return outputs_list

def get_predictions(model, test_loader, device="cuda"):
    sfm = nn.Softmax()
    predictions = list()
    for batch_idx, inputs in enumerate(test_loader):
        inputs = inputs.type(torch.FloatTensor).to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            predictions.append(sfm(outputs)) ## ADD SOFTMAX
    predictions = np.concatenate([t.cpu().numpy() for t in predictions])
    return predictions

In [21]:
# model_ft.load_state_dict(torch.load(f"tmp/{tmp_folder_name}/best_run_{FOLD}.pth"))

In [22]:
n_classes = len(label2code)
device = torch.device("cuda")

results_folder = "resnet50_aug_mixup_repr_folds"
# os.mkdir(f"tmp/{results_folder}")
BS = 32
for FOLD in range(N_FOLDS):
    print("PROCESSING FOLD", FOLD)
    
    model_ft = models.resnet50(pretrained=True)
    model_ft.fc = nn.Sequential(
        nn.Linear(model_ft.fc.in_features, n_classes)
    )
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss(reduction="sum")

    optimizer_ft = optim.Adam(model_ft.parameters(), lr=2e-4)
    lr_scheduler = ReduceLROnPlateau(optimizer_ft, 'min', patience=4, factor=0.5, verbose=True, min_lr=2e-5)

    train_loader = torch.utils.data.DataLoader(sdf_train_list[FOLD], batch_size=BS, shuffle=True,
                                               num_workers=6, drop_last=False, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(sdf_val_list[FOLD], batch_size=BS, drop_last=False, pin_memory=True,
                                        num_workers=6)
    
    # full train cycle with one fold for cross-val:
    model_ft = train_on_fold(
        model_ft,
        train_loader,
        val_loader,
        FOLD,
        criterion,
        optimizer_ft,
        lr_scheduler,
        results_folder,
        None
    )
    
    # predict from one model:
    preds = predict_on_val(
        model_ft,
        val_loader
    )
    val_folds[FOLD]["preds"] = [code2label[c] for c in preds]
    
    # Save val predictions from one model:
    val_out = get_val_outputs(model_ft, val_loader)
    gt = val_folds[FOLD].label.map(label2code).values
    val_losses = list()
    
    for idx in range(len(gt)):
        item_loss = criterion(torch.Tensor([val_out[idx]]), torch.LongTensor([gt[idx]])).numpy()
        val_losses.append(item_loss)
    val_losses = np.array(val_losses)
    val_folds[FOLD]["loss"] = val_losses
    val_folds[FOLD].reset_index(drop=True).sort_values(by="loss", ascending=True).to_csv(
        f"tmp/{results_folder}/val_loss_{FOLD}.csv", index=False
    )
    
    ## for debug, save predictions for train as well:
    ### debug train audios:
    train_debug_loader = torch.utils.data.DataLoader(sdf_train_list[FOLD], batch_size=BS, shuffle=False,
                                               num_workers=6, drop_last=False, pin_memory=True)
    train_out = get_val_outputs(model_ft, train_debug_loader)
    gt = train_folds[FOLD].label.map(label2code).values
    train_losses = list()

    for idx in range(len(gt)):
        item_loss = criterion(torch.Tensor([train_out[idx]]), torch.LongTensor([gt[idx]])).numpy()
        train_losses.append(item_loss)

    train_losses = np.array(train_losses)
    train_folds[FOLD]["loss"] = train_losses
    train_folds[FOLD]["preds"] = [code2label[c] for c in np.argmax(train_out, axis=1)]
    train_folds[FOLD].reset_index(drop=True).sort_values(by="loss", ascending=False).to_csv(
        f"tmp/{results_folder}/train_loss_{FOLD}.csv", index=False
    )
    
    ## MAKE SUBMISSION (from one model):
    sample_subm = pd.read_csv("Submission1.csv")
    sample_subm["image_fn"] = sample_subm.fn.apply(get_image_path)
    subm_dataset = SpectrogramTestDataset([[path, None] for path in sample_subm.image_fn.values ], conf)
    subm_loader = torch.utils.data.DataLoader(subm_dataset, batch_size=16)

    preds = get_predictions(model_ft, subm_loader)

    for c in sample_subm.columns:
        if c in {"fn", "image_fn"}:
            continue
        c_idx = label2code[c]
        sample_subm[c] = preds[:, c_idx]

    sample_subm.drop("image_fn", axis=1).to_csv(f'tmp/{results_folder}/subm_{FOLD}.csv', index=False)

PROCESSING FOLD 0
Train Epoch: 0 [0/4258 (0%)]	Loss: 172.038345

Test set: Average loss: 4.6556, Accuracy: 24/451 (5%)

Training so far 1.7080100456873575 minutes
Train Epoch: 1 [0/4258 (0%)]	Loss: 148.428726

Test set: Average loss: 3.7117, Accuracy: 59/451 (13%)

Training so far 3.382240867614746 minutes
Train Epoch: 2 [0/4258 (0%)]	Loss: 141.543137

Test set: Average loss: 2.8740, Accuracy: 127/451 (28%)

Training so far 5.105073436101278 minutes
Train Epoch: 3 [0/4258 (0%)]	Loss: 121.851776

Test set: Average loss: 2.5296, Accuracy: 170/451 (38%)

Training so far 6.736630479494731 minutes
Train Epoch: 4 [0/4258 (0%)]	Loss: 125.744942

Test set: Average loss: 2.5422, Accuracy: 169/451 (37%)

Training so far 8.372699813048046 minutes
Train Epoch: 5 [0/4258 (0%)]	Loss: 50.592739

Test set: Average loss: 1.7072, Accuracy: 272/451 (60%)

Training so far 10.03559097846349 minutes
Train Epoch: 6 [0/4258 (0%)]	Loss: 36.970451

Test set: Average loss: 1.4853, Accuracy: 295/451 (65%)

Traini


Test set: Average loss: 0.5439, Accuracy: 397/451 (88%)

Training so far 57.820076155662534 minutes
Train Epoch: 49 [0/4258 (0%)]	Loss: 39.251205

Test set: Average loss: 0.5314, Accuracy: 394/451 (87%)

Training so far 58.81813518206278 minutes
Train Epoch: 50 [0/4258 (0%)]	Loss: 4.157947

Test set: Average loss: 0.5339, Accuracy: 397/451 (88%)

Training so far 59.805141270160675 minutes
Train Epoch: 51 [0/4258 (0%)]	Loss: 8.452828

Test set: Average loss: 0.5477, Accuracy: 393/451 (87%)

Training so far 60.807098321119945 minutes
Train Epoch: 52 [0/4258 (0%)]	Loss: 0.558549

Test set: Average loss: 0.5112, Accuracy: 398/451 (88%)

Training so far 61.79184664487839 minutes
Train Epoch: 53 [0/4258 (0%)]	Loss: 0.246121

Test set: Average loss: 0.5312, Accuracy: 398/451 (88%)

Training so far 62.77208450635274 minutes
Train Epoch: 54 [0/4258 (0%)]	Loss: 15.258808

Test set: Average loss: 0.5318, Accuracy: 401/451 (89%)

Training so far 63.744934697945915 minutes
Train Epoch: 55 [0/4258 

/home/letfoolsdie/virtual_envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/letfoolsdie/virtual_envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


PROCESSING FOLD 1
Train Epoch: 0 [0/4258 (0%)]	Loss: 170.355942

Test set: Average loss: 4.7667, Accuracy: 26/451 (6%)

Training so far 0.9675414681434631 minutes
Train Epoch: 1 [0/4258 (0%)]	Loss: 141.894745

Test set: Average loss: 3.5472, Accuracy: 78/451 (17%)

Training so far 1.9283783237139385 minutes
Train Epoch: 2 [0/4258 (0%)]	Loss: 98.352989

Test set: Average loss: 2.7103, Accuracy: 149/451 (33%)

Training so far 2.9087892333666483 minutes
Train Epoch: 3 [0/4258 (0%)]	Loss: 105.105301

Test set: Average loss: 2.4682, Accuracy: 193/451 (43%)

Training so far 3.8816919763882956 minutes
Train Epoch: 4 [0/4258 (0%)]	Loss: 70.317078

Test set: Average loss: 1.9149, Accuracy: 248/451 (55%)

Training so far 4.8583781560262045 minutes
Train Epoch: 5 [0/4258 (0%)]	Loss: 57.089363

Test set: Average loss: 1.6211, Accuracy: 288/451 (64%)

Training so far 5.82958261569341 minutes
Train Epoch: 6 [0/4258 (0%)]	Loss: 39.443642

Test set: Average loss: 1.6764, Accuracy: 274/451 (61%)

Train


Test set: Average loss: 0.7213, Accuracy: 387/451 (86%)

Training so far 47.63714360793431 minutes
Train Epoch: 49 [0/4258 (0%)]	Loss: 7.981059

Test set: Average loss: 0.7046, Accuracy: 390/451 (86%)

Training so far 48.61775028308232 minutes
Train Epoch: 50 [0/4258 (0%)]	Loss: 0.382614

Test set: Average loss: 0.6903, Accuracy: 384/451 (85%)

Training so far 49.60714798768361 minutes
Train Epoch: 51 [0/4258 (0%)]	Loss: 0.265543

Test set: Average loss: 0.7037, Accuracy: 388/451 (86%)

Training so far 50.61866446336111 minutes
Train Epoch: 52 [0/4258 (0%)]	Loss: 51.378540

Test set: Average loss: 0.7044, Accuracy: 381/451 (84%)

Training so far 51.60544813474019 minutes
Train Epoch: 53 [0/4258 (0%)]	Loss: 69.463409

Test set: Average loss: 0.7043, Accuracy: 387/451 (86%)

Training so far 52.58300413688024 minutes
Train Epoch: 54 [0/4258 (0%)]	Loss: 27.468594

Test set: Average loss: 0.7268, Accuracy: 389/451 (86%)

Training so far 53.552595523993176 minutes
Train Epoch: 55 [0/4258 (0


Test set: Average loss: 1.0255, Accuracy: 348/451 (77%)

Training so far 21.58089808622996 minutes
Train Epoch: 22 [0/4258 (0%)]	Loss: 7.753327

Test set: Average loss: 1.0149, Accuracy: 353/451 (78%)

Training so far 22.567681992053984 minutes
Train Epoch: 23 [0/4258 (0%)]	Loss: 7.176729

Test set: Average loss: 0.9204, Accuracy: 365/451 (81%)

Training so far 23.547632376352947 minutes
Train Epoch: 24 [0/4258 (0%)]	Loss: 32.886639

Test set: Average loss: 1.0416, Accuracy: 353/451 (78%)

Training so far 24.521318459510802 minutes
Train Epoch: 25 [0/4258 (0%)]	Loss: 24.747412

Test set: Average loss: 1.0447, Accuracy: 349/451 (77%)

Training so far 25.492129472891488 minutes
Train Epoch: 26 [0/4258 (0%)]	Loss: 38.221863

Test set: Average loss: 0.8925, Accuracy: 360/451 (80%)

Training so far 26.483438154061634 minutes
Train Epoch: 27 [0/4258 (0%)]	Loss: 102.722504

Test set: Average loss: 1.0416, Accuracy: 353/451 (78%)

Training so far 27.45859580039978 minutes
Train Epoch: 28 [0/4

Train Epoch: 4 [0/4258 (0%)]	Loss: 0.689750

Test set: Average loss: 0.7377, Accuracy: 383/451 (85%)

Training so far 68.34244406620661 minutes
Train Epoch: 5 [0/4258 (0%)]	Loss: 0.355623

Test set: Average loss: 0.7261, Accuracy: 386/451 (86%)

Training so far 69.30972306728363 minutes
Train Epoch: 6 [0/4258 (0%)]	Loss: 0.722595

Test set: Average loss: 0.7243, Accuracy: 382/451 (85%)

Training so far 70.28663562138875 minutes
Train Epoch: 7 [0/4258 (0%)]	Loss: 0.491879

Test set: Average loss: 0.7205, Accuracy: 387/451 (86%)

Training so far 71.26583449045818 minutes
Train Epoch: 8 [0/4258 (0%)]	Loss: 1.451376

Test set: Average loss: 0.7502, Accuracy: 384/451 (85%)

Training so far 72.23256327708562 minutes
Train Epoch: 9 [0/4258 (0%)]	Loss: 0.143068

Test set: Average loss: 0.7463, Accuracy: 387/451 (86%)

Training so far 73.21324704885483 minutes
time spent training: 73.2132474541664 minutes
BEST LOSS: 0.7204920280799104
BEST ACC: 85.80931263858093


In [28]:
[print(i/100) for i in [88.69179600886918,85.80931263858093,85.80931263858093]]

0.8869179600886917
0.8580931263858094
0.8580931263858094


[None, None, None]

In [25]:
## MAKE ONE SUBMISSION FROM ALL SUBS:
import pandas as pd
import numpy as np

s = f'tmp/{results_folder}/'
preds_to_average = [
    f"subm_{f}.csv"
    for f in range(N_FOLDS)
]

all_subs = list()
source_pred = pd.read_csv(s + preds_to_average[0])
pred_cols = source_pred.drop("fn", axis=1).columns.values
for file in preds_to_average[1:]:
    tmp = pd.read_csv(s + file)
    source_pred[pred_cols] += tmp[pred_cols]

source_pred[pred_cols] /= len(preds_to_average)

In [26]:
np.all(np.isclose((source_pred[pred_cols].sum(axis=1)).values, np.ones(source_pred.shape[0])))

True

In [27]:
source_pred.to_csv(f'tmp/{results_folder}/{results_folder}_merged.csv', index=False)